In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cars-datasets-2025/Cars Datasets 2025.csv


In [3]:
data = pd.read_csv('/kaggle/input/cars-datasets-2025/Cars Datasets 2025.csv', encoding= 'latin1')
data.head()

,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
0,FERRARI,SF90 STRADALE,V8,3990 cc,963 hp,340 km/h,2.5 sec,"$1,100,000",plug in hyrbrid,2,800 Nm
1,ROLLS ROYCE,PHANTOM,V12,6749 cc,563 hp,250 km/h,5.3 sec,"$460,000",Petrol,5,900 Nm
2,Ford,KA+,1.2L Petrol,"1,200 cc",70-85 hp,165 km/h,10.5 sec,"$12,000-$15,000",Petrol,5,100 - 140 Nm
3,MERCEDES,GT 63 S,V8,"3,982 cc",630 hp,250 km/h,3.2 sec,"$161,000",Petrol,4,900 Nm
4,AUDI,AUDI R8 Gt,V10,"5,204 cc",602 hp,320 km/h,3.6 sec,"$253,290",Petrol,2,560 Nm


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Company Names              1218 non-null   object
 1   Cars Names                 1218 non-null   object
 2   Engines                    1218 non-null   object
 3   CC/Battery Capacity        1215 non-null   object
 4   HorsePower                 1218 non-null   object
 5   Total Speed                1218 non-null   object
 6   Performance(0 - 100 )KM/H  1212 non-null   object
 7   Cars Prices                1218 non-null   object
 8   Fuel Types                 1218 non-null   object
 9   Seats                      1218 non-null   object
 10  Torque                     1217 non-null   object
dtypes: object(11)
memory usage: 104.8+ KB


In [5]:
data.describe()

,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
count,1218,1218,1218,1215,1218,1218,1212,1218,1218,1218,1217
unique,37,1201,356,311,456,114,180,535,23,19,263
top,Nissan,Macan T,I4,"2,000 cc",355 hp,250 km/h,6.5 sec,"$35,000",Petrol,5,400 Nm
freq,149,2,64,31,23,145,45,36,871,692,72


In [6]:

import numpy as np
import re

def parse_price(val):
    if pd.isna(val):
        return np.nan
    
    val = str(val).strip()  # quitar espacios al inicio y final
    
    # Manejar rangos con '-' o '/' tomando promedio
    if '-' in val or '/' in val:
        val = val.replace('/', '-')
        parts = val.split('-')
        nums = []
        for p in parts:
            # Extraer solo dígitos y punto
            match = re.findall(r'\d+\.?\d*', p)
            if match:
                nums.append(float(match[0]))
        if len(nums) == 2:
            return sum(nums)/2
        elif len(nums) == 1:
            return nums[0]
        else:
            return np.nan
    else:
        # Extraer primer número válido
        match = re.findall(r'\d+\.?\d*', val)
        if match:
            return float(match[0])
        else:
            return np.nan

# Aplicar a la columna
data['Cars Prices'] = data['Cars Prices'].apply(parse_price)

# Convertir Seats a int, manejando posibles valores no numéricos
data['Seats'] = pd.to_numeric(data['Seats'], errors='coerce').astype('Int64')

#Convertir Total Speed a int, manejando posibles valores no numéricos
data["Total Speed"] = (
    data["Total Speed"]
    .astype(str)
    .str.extract(r"(\d+\.?\d*)")   # captura números con decimales si los hay
    .astype(float)                 # convierte a float
)

import pandas as pd
import re

# clean Torque
def clean_torque(value):
    nums = re.findall(r"\d+", str(value))
    if not nums:
        return pd.NA
    nums = list(map(int, nums))
    return int(sum(nums) / len(nums))  # promedio si hay rango

data["Torque"] = data["Torque"].apply(clean_torque).astype("Int64")

# clean HorsePower
data["HorsePower"] = (
    data["HorsePower"]
    .astype(str)
    .str.extract(r"(\d+\.?\d*)", expand=False)
    .astype(float)      # primero float
    .round(0)           # redondea si tiene decimales
    .astype("Int64")    # ahora seguro a entero
)

# clean CC/Battery Capacity
data["CC/Battery Capacity"] = (
    data["CC/Battery Capacity"]
    .astype(str)
    .str.extract(r"(\d+\.?\d*)", expand=False)
    .astype(float)
    .round(0)
    .astype("Int64")
)

# clean Performance(0-100)
data["Performance(0 - 100 )KM/H"] = (
    data["Performance(0 - 100 )KM/H"]
    .astype(str)
    .str.extract(r"(\d+\.?\d*)", expand=False)
    .astype(float)
    .round(0)
    .astype("Int64")
)


data.head()

,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
0,FERRARI,SF90 STRADALE,V8,3990,963,340.0,2,1.0,plug in hyrbrid,2,800
1,ROLLS ROYCE,PHANTOM,V12,6749,563,250.0,5,460.0,Petrol,5,900
2,Ford,KA+,1.2L Petrol,1,70,165.0,10,13.5,Petrol,5,120
3,MERCEDES,GT 63 S,V8,3,630,250.0,3,161.0,Petrol,4,900
4,AUDI,AUDI R8 Gt,V10,5,602,320.0,4,253.0,Petrol,2,560


In [7]:
top_index = data['Engines'].value_counts(ascending=False).head(30).index

data_top = data[data['Engines'].isin(top_index)]
#data_top['HorsePower'].value_counts().head(30).sum()
data_top['Total Speed'].nunique()

76

In [8]:
data_top = data_top.drop(['Cars Names'], axis=1)

In [9]:
data_top.head()

,Company Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
0,FERRARI,V8,3990,963,340.0,2,1.0,plug in hyrbrid,2,800
1,ROLLS ROYCE,V12,6749,563,250.0,5,460.0,Petrol,5,900
3,MERCEDES,V8,3,630,250.0,3,161.0,Petrol,4,900
4,AUDI,V10,5,602,320.0,4,253.0,Petrol,2,560
5,BMW,V8,3,710,341.0,3,499.0,Petrol,2,770


## First Aproach:

In [10]:
from sklearn.model_selection import train_test_split

X = data_top.drop(['Cars Prices'], axis=1)
y = data_top['Cars Prices']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8)


In [11]:
X.describe()

,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Seats,Torque
count,560.0,562.0,562.000000,561.0,557.0,562.0
mean,1342.575,346.355872,230.969751,6.666667,4.798923,460.743772
std,2033.343553,249.218818,55.759294,2.705374,1.466336,263.442076
min,1.0,64.0,100.000000,2.0,1.0,95.0
25%,2.0,168.5,191.250000,5.0,4.0,264.0
50%,6.0,290.5,220.000000,6.0,5.0,400.0
75%,1999.0,443.0,250.000000,8.0,5.0,600.0
max,7993.0,1850.0,500.000000,18.0,8.0,1600.0


In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import FunctionTransformer

categorical_cols = ['Company Names', 'Fuel Types', 'Engines']

transform_cat = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
])


transformer = ColumnTransformer(transformers=[
    ('categorical', transform_cat, categorical_cols)
])

model = RandomForestRegressor(n_estimators = 100, random_state=0)

mypipeline1 = Pipeline(steps=[
    ('preprocessor', transformer),
    ('model', model)
])

In [13]:
from sklearn.metrics import mean_absolute_error

mypipeline1.fit(X_train, y_train)
preds = mypipeline1.predict(X_valid)

mae_score = mean_absolute_error(y_valid, preds)

#afegint el freq_encoder surt un MAE més elevat, vol dir 
#que no és gaire eficaç aquesta tèncica en aquest cas

print(f'MAE Score: {mae_score.round(4)}')

MAE Score: 21.4956


## Second Aproach (Cross-Validation):

In [26]:
data.head(30)

,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
0,FERRARI,SF90 STRADALE,V8,3990,963,340.0,2,1.0,plug in hyrbrid,2,800
1,ROLLS ROYCE,PHANTOM,V12,6749,563,250.0,5,460.0,Petrol,5,900
2,Ford,KA+,1.2L Petrol,1,70,165.0,10,13.5,Petrol,5,120
3,MERCEDES,GT 63 S,V8,3,630,250.0,3,161.0,Petrol,4,900
4,AUDI,AUDI R8 Gt,V10,5,602,320.0,4,253.0,Petrol,2,560
5,BMW,Mclaren 720s,V8,3,710,341.0,3,499.0,Petrol,2,770
6,ASTON MARTIN,VANTAGE F1,V8,3,656,314.0,4,193.0,Petrol,2,685
7,BENTLEY,Continental GT Azure,V8,3,550,318.0,4,311.0,Petrol,4,900
8,LAMBORGHINI,VENENO ROADSTER,V12,6,750,356.0,3,4.0,Petrol,2,690
9,FERRARI,F8 TRIBUTO,V8,3,710,340.0,3,280.0,Petrol,2,770


In [27]:
from sklearn.model_selection import cross_val_score, KFold

OH_encoder = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown = 'ignore', sparse_output=False))
])

model = RandomForestRegressor(n_estimators=200, random_state=0)

preprocessor = ColumnTransformer(transformers=[
    ('onehot', OH_encoder, categorical_cols)
])

mypipeline2 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])


kf = KFold(n_splits=5, shuffle=True, random_state=0)

mae_score = -1*cross_val_score(mypipeline2, X, y, cv=kf, scoring='neg_mean_absolute_error')

print('MAE score: \n', mae_score)

MAE score: 
 [20.67786945 26.94275973 20.62817182 15.3508413  18.97971125]


In [28]:
print(mae_score.mean())

20.51587071223014


Amb aquest segon aproach aplicant corss-validation i un shuffle aleatori al dataset aconseguim un MAE més baix